# Ordena todos los productos en la web de Trust por menor a mayor precio de la unidad

In [ ]:
# Instalaciones

import requests
import pandas as pd
import random
import numpy as np
import time
from google.colab import files
%pip install bs4
from bs4 import BeautifulSoup
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
options = webdriver.ChromeOptions() # Usamos chrome, se podria usar otro.
options.add_argument('--headless') # Chromium sin interfaz grafica
options.add_argument('--no-sandbox') # Seguridad
options.add_argument('--disable-dev-shm-usage') # configuracion de linux
options.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""') # user agent
import xlrd
from lxml import etree
!pip install tika
from tika import parser
from google.colab import files
from time import sleep
import re
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
nuestros_headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
    }



# Precio venta dolar santander + $10
url = 'https://www.google.com/search?q=dolar+santander+hoy+venta+argentina&rlz=1C1CHBF_esAR973AR973&oq=d&aqs=chrome.1.69i57j35i39j69i60l6.6018j0j4&sourceid=chrome&ie=UTF-8'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
dolar_hoy = soup.find_all('td')
dolar_hoy = [dolar.text for dolar in dolar_hoy]
dolar_hoy = float([dolar for dolar in dolar_hoy if '$' in dolar][-1].split(' ')[1].replace(',','.')) + 10





 # PRIMERA BUSQUEDA
# Definicion de los links
urls = ['http://trust.kuder.com.ar/','http://wtg.kuder.com.ar/','http://star-nutrition.kuder.com.ar/']

# Variables finales
nombresFinal = []
preciosFinal = []
linksFinal = []


#driver = webdriver.Chrome('chromedriver',options=options)
for iter in range(len(urls)):
  try:
    driver = webdriver.Chrome('chromedriver',options=options)
    # Hacemos que vaya a la url
    driver.get(urls[iter])
    sleep(1.2)
    input = driver.find_element_by_xpath('//*[@id="btnUsuario_mobile"]')
    input.click()

    # Usuario
    input = driver.find_element_by_xpath('//*[@id="usuario_login"]')
    sleep(1)
    input.send_keys("20412003382")

    # Contra
    input = driver.find_element_by_xpath('//*[@id="clave_login"]')
    sleep(1)
    input.send_keys("unabala9898")

    # entrar
    driver.find_element_by_xpath('//*[@id="btnLogin"]').click()
    sleep(2)


    # Click al menu desplegable
    driver.find_element_by_xpath('//*[@id="mainNav"]/div[3]/nav/button/span').click()
    sleep(2)

    # Click en productos
    driver.find_element_by_xpath('//*[@id="8_2"]').click()
    sleep(1)


    # Click en ver todos
    driver.find_element_by_xpath("//a[@class='dropdown-item dropdown-item-todos'][contains(text(),'TODOS')]").click()
    sleep(1.5)


    # Creacion var
    lista_nombres = []
    lista_links = []
    lista_precios = []

    # cantidad de "ver Mas"
    paginas = driver.find_elements_by_xpath('//*[@id="paginado"]/li//a')

    for i in range(len(paginas)):
      sleep(3)
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      sleep(1)
      # Nombres
      nombres = driver.find_elements_by_xpath("//h4[@class='titulo-articulo pt-4']")
      nombres = [nombre.text for nombre in nombres]
      lista_nombres.extend(nombres)

      # Links
      links = driver.find_elements_by_css_selector('a')
      links = [link.get_attribute('href') for link in links]
      links = [link for link in links if 'id=' in str(link)]
      lista_links.extend(links)

      # Precios
      precios = driver.find_elements_by_xpath('//strong[contains(text(),"$")]')
      precios = [precio.text for precio in precios]
      # transformar a int + si esta en dolares pasarlo a pesos
      preciosCaja = []
      for pre in precios:
        listo = False
        pre =pre.replace('.','')
        while listo == False:
          try:
            int(pre)
            listo = True
          except:
            pre = pre[1:]
        if iter != 2:
          preciosCaja.append(round(int(pre)*dolar_hoy))
        else:
           preciosCaja.append(int(pre))
      lista_precios.extend(preciosCaja)

      


      # Click en ver mas
      driver.get(urls[iter] + 'listado.html?pagina=' + str(i+2))
    nombresFinal.extend(lista_nombres)  
    preciosFinal.extend(lista_precios)  
    linksFinal.extend(lista_links) 

  except:
    print(f'No se pudo link {urls[iter]}')
    continue
 

# df
df1 = pd.DataFrame(np.array([nombresFinal,preciosFinal,linksFinal]).T, columns=['Producto','Inversion','Link'])





# Iterar por links para obtener datos.

# Marca
cantidadAlternativa = []
driver = webdriver.Chrome('chromedriver',options=options)
marcaWeb = []
for linkIter in range(len(linksFinal)):
  encontrado = ''
  driver.get(linksFinal[linkIter])
  sleep(1.5)
  descCantidad = driver.find_elements_by_xpath('//*[@id="ficha-articulo"]/div[3]')

  
  # Cantidad alternativa
  try:
    descCantidad = descCantidad[0].split('Caja x ')[-1].split('U')[0].replace(' ','')
    cantidadAlternativa.append(int(descCantidad))
  except:
    cantidadAlternativa.append(99)
  print(f'Iteracion nro {linkIter+1} de {len(linksFinal)}')
    
    


# Cantidad
reglas_de_busqueda = ['X(\d+)$','X\s(\d+)$','(\d+)U','(\d+)\sU']
cantidad = [None] * len(nombresFinal)

for reg in reglas_de_busqueda:
  titles = [re.findall(reg,m) for m in nombresFinal]
  for i1 in range(len(titles)):
    if cantidad[i1] == None or cantidad[i1] == []:
      cantidad[i1] = titles[i1]

finalCantidad = []
for i in range(len(cantidad)):
  try:
    finalCantidad.append(int(cantidad[i][0]))
  except:
    finalCantidad.append(cantidadAlternativa[i])
df2 = pd.DataFrame(np.array([finalCantidad]).T, columns=['Unidades'])

# Join de los dfs
df = df1.join(df2)

# Emprolijado
df['Inversion'] = df['Inversion'].astype(int)
df['Unidades'] = df['Unidades'].astype(int)
for i in range(len(df)):
  if 'AMINO ENERGY' in df.Producto[i] or 'ON BCAA 5000' in df.Producto[i]:
    df.Unidades[i] = 6
  elif 'CAJA SYNTHA 6 EDGE X 3.77LB' in df.Producto[i]:
    df.Unidades[i] = 4
  elif 'CREA MICRONIZED X300 GR' in df.Producto[i] or 'C4 ORIGINAL WATER 390GRS' in df.Producto[i]:
    df.Unidades[i] = 12
df['Precio_Unidad'] = round(df['Inversion']/df['Unidades'])

# Reubicar columnas
Producto = df.pop('Producto')
Inversion = df.pop('Inversion')
Link = df.pop('Link')
Unidades = df.pop('Unidades')
Precio_Unidad = df.pop('Precio_Unidad')

df.insert(0, 'Producto', Producto)
df.insert(1, 'Precio_Unidad', Precio_Unidad)
df.insert(2, 'Inversion', Inversion)
df.insert(3, 'Unidades', Unidades)
df.insert(4, 'Link', Link)

# Ordenar por precio
df = df.sort_values('Precio_Unidad')
df.head(60)

  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 2s (109 kB/s)
Reading package lis

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:78: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:92: DeprecationWarning: find_element_by

Iteracion nro 1 de 187
Iteracion nro 2 de 187
Iteracion nro 3 de 187
Iteracion nro 4 de 187
Iteracion nro 5 de 187
Iteracion nro 6 de 187
Iteracion nro 7 de 187
Iteracion nro 8 de 187
Iteracion nro 9 de 187
Iteracion nro 10 de 187
Iteracion nro 11 de 187
Iteracion nro 12 de 187
Iteracion nro 13 de 187
Iteracion nro 14 de 187
Iteracion nro 15 de 187
Iteracion nro 16 de 187
Iteracion nro 17 de 187
Iteracion nro 18 de 187
Iteracion nro 19 de 187
Iteracion nro 20 de 187
Iteracion nro 21 de 187
Iteracion nro 22 de 187
Iteracion nro 23 de 187
Iteracion nro 24 de 187
Iteracion nro 25 de 187
Iteracion nro 26 de 187
Iteracion nro 27 de 187
Iteracion nro 28 de 187
Iteracion nro 29 de 187
Iteracion nro 30 de 187
Iteracion nro 31 de 187
Iteracion nro 32 de 187
Iteracion nro 33 de 187
Iteracion nro 34 de 187
Iteracion nro 35 de 187
Iteracion nro 36 de 187
Iteracion nro 37 de 187
Iteracion nro 38 de 187
Iteracion nro 39 de 187
Iteracion nro 40 de 187
Iteracion nro 41 de 187
Iteracion nro 42 de 187
I

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Producto,Precio_Unidad,Inversion,Unidades,Link
162,CAJA SNACK PROT BAR AVE X 108U,142.0,15390,108,http://star-nutrition.kuder.com.ar/ficha.html?...
163,CAJA SNACK PROT BAR BAN X108 U,142.0,15390,108,http://star-nutrition.kuder.com.ar/ficha.html?...
61,CAJA GLUTAMINE DRIVE X 300GRS.,240.0,23801,99,http://wtg.kuder.com.ar/ficha.html?id=293427
72,CAJA MUSCLE INF. VANILLA X 2LB,305.0,30148,99,http://wtg.kuder.com.ar/ficha.html?id=293429
70,CAJA MUSCLE INF. CHOCO. X 2LB,305.0,30148,99,http://wtg.kuder.com.ar/ficha.html?id=293430
54,CAJA BCAA6000 WATERM.X 255GR.,321.0,31735,99,http://wtg.kuder.com.ar/ficha.html?id=293426
53,CAJA BCAA6000 F.PUNCH X 255GR.,321.0,31735,99,http://wtg.kuder.com.ar/ficha.html?id=293425
66,CAJA LIPO6 B.T.W.GRAPE X 240GR,353.0,34909,99,http://wtg.kuder.com.ar/ficha.html?id=293424
65,CAJA LIPO6 B.T. BLUE RAZZX240G,353.0,34909,99,http://wtg.kuder.com.ar/ficha.html?id=293423
73,CAJA MUSCLE INF. VANILLA X 5LB,401.0,39669,99,http://wtg.kuder.com.ar/ficha.html?id=293431


In [ ]:
# <----- Apretar Play para descargarlo como excel
df.to_csv('Ganancias_Trust.csv')
files.download("Ganancias_Trust.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>